In [1]:
import pandas as pd
import numpy as np

# Data preparation

## Initial data

In [2]:
df = pd.read_csv("Prozorro data/Part_0_Tenders.csv", sep=";", dtype={"IDOrganizator":str}, parse_dates=["Дата"])
df.dropna(inplace=True)
df.columns = ["tender.id", "tender.procurementMethod", "tender.date", "tender.value.amount", "tender.items.classification", 
              "tender.mainProcurementCategory", "tender.procuringEntity.identifier.legalName", "tender.procuringEntity.identifier.id", 
              "num_contr_in_tender", "tender.lot.value.amount", "contracts.value.amount"]

df = df[df["tender.mainProcurementCategory"]!="Undefined"]

df["contracts.value.amount"] = df["contracts.value.amount"].str.replace(",", ".").astype(float)
df["tender.value.amount"] = df["tender.value.amount"].str.replace(",", ".").astype(float)
df["tender.lot.value.amount"] = df["tender.lot.value.amount"].str.replace(",", ".").astype(float)

print("The shape of the DF: ", df.shape)
df.head(3)

The shape of the DF:  (521766, 11)


,tender.id,tender.procurementMethod,tender.date,tender.value.amount,tender.items.classification,tender.mainProcurementCategory,tender.procuringEntity.identifier.legalName,tender.procuringEntity.identifier.id,num_contr_in_tender,tender.lot.value.amount,contracts.value.amount
1,UA-2016-07-15-000064-c,limited,2020-08-26,8000.0,"09000000-3 Нафтопродукти, паливо, електроенерг...",Goods,Гайсинський медичний фаховий коледж,02011019,1,8000.0,7965.0
2,UA-2018-12-29-001589-b,limited,2019-02-01,27358.0,"09000000-3 Нафтопродукти, паливо, електроенерг...",Goods,"КОМУНАЛЬНИЙ ЗАКЛАД КУЛЬТУРИ ""ДНІПРОВСЬКИЙ БУДИ...",37621184,1,27358.0,27358.0
3,UA-2018-12-29-001061-c,limited,2019-02-01,8663.0,"09000000-3 Нафтопродукти, паливо, електроенерг...",Goods,"МІСЬКИЙ КОМУНАЛЬНИЙ ЗАКЛАД КУЛЬТУРИ ""ДНІПРОВСЬ...",02222621,1,8663.0,8663.0


In [3]:
df["tender.id"].nunique()

521766

In [4]:
## Adding the entity type

df_entities = pd.read_excel("Additional data/GPA Довідник.xlsx", sheet_name="Master sheet_All in one", dtype={"EDRPOU":str})
df_entities.drop(["Google translate", "Correct translation"], axis=1, inplace=True)

df = pd.merge(df, df_entities[["EDRPOU", "GPA Kind"]], left_on= "tender.procuringEntity.identifier.id", right_on="EDRPOU", how="left")
df.drop("EDRPOU", axis=1, inplace=True)
df["GPA Kind"].fillna("non-GPA entity", inplace=True)
df = df[df["GPA Kind"]!="non-GPA entity"] ### SIMPLIFY LATER

## Add thresholds
df_thresholds = pd.read_excel("Additional data/Thresholds.xlsx")

df = pd.merge(df, df_thresholds, left_on=["GPA Kind", "tender.mainProcurementCategory"], right_on=["GPA Kind", "CPV type"], how="left")
df.drop("CPV type", axis=1, inplace=True)

## Exchange rate
df_exchange_sdr = pd.read_excel("Additional data/Офіційний курс гривні щодо іноземних валют.xlsx", parse_dates=["Дата"])
df_exchange_sdr["ex_rate"] = df_exchange_sdr["Офіційний курс гривні, грн"] / df_exchange_sdr["Кількість одиниць"]

df = pd.merge(df, df_exchange_sdr[["Дата", "ex_rate"]], left_on= "tender.date", right_on="Дата", how="left")
df.drop("Дата", axis=1, inplace=True)

df["tender.value.amount_SDR"] = df["tender.value.amount"] / df["ex_rate"]
df["contracts.value.amount_SDR"] = df["contracts.value.amount"] / df["ex_rate"]
df["tender.lot.value.amount_SDR"] = df["tender.lot.value.amount"] / df["ex_rate"]

## Finilizing
df = df[df["tender.value.amount_SDR"]>=df["SDR Threshold"]]

df = df[['tender.id', 'tender.date', 'tender.procurementMethod',
       'tender.items.classification', 'tender.mainProcurementCategory',
       'tender.procuringEntity.identifier.legalName',
       'tender.procuringEntity.identifier.id', 'GPA Kind',
       "tender.value.amount_SDR", "tender.lot.value.amount_SDR", 'contracts.value.amount_SDR', 
       "num_contr_in_tender"]]

df.rename({"GPA Kind":"GPA entity type"}, axis=1, inplace =True)

print("The shape of the DataFrame: ", df.shape)
df.head(3)

The shape of the DataFrame:  (7435, 12)


,tender.id,tender.date,tender.procurementMethod,tender.items.classification,tender.mainProcurementCategory,tender.procuringEntity.identifier.legalName,tender.procuringEntity.identifier.id,GPA entity type,tender.value.amount_SDR,tender.lot.value.amount_SDR,contracts.value.amount_SDR,num_contr_in_tender
0,UA-2018-12-05-002348-c.2,2019-03-21,selective,45000000-7 Будівельні роботи та поточний ремонт,Works,Служба автомобільних доріг у Полтавській області,25898491,Central government entities,1.795485e+07,1.795485e+07,1.662194e+07,1
4,UA-2018-12-26-003794-b,2019-11-01,limited,64000000-6 Поштові та телекомунікаційні послуги,Services,Управління поліції охорони в Запорізькій області,40108947,Central government entities,1.319491e+05,1.319491e+05,1.319491e+05,5
5,UA-2018-12-28-000557-a,2019-08-01,limited,"09000000-3 Нафтопродукти, паливо, електроенерг...",Goods,Управління освіти Дніпровської районної в міст...,37397216,Subcentral government entities,3.634378e+06,3.634378e+06,3.634378e+06,9


In [5]:
GPA_tenders_list = df["tender.id"].unique()

In [6]:
df["tender.id"].nunique()

7435

In [7]:
df["num_contr_in_tender"].sum()

12016

In [8]:
df.to_excel("df_basic_data.xlsx", index=False)

## Tenders and Participants

In [130]:
df = pd.read_csv("Prozorro data/Part_1_Tenders_Participants.csv", sep=";", dtype={"IDOrganizator":str, "IDTenderer":str}, parse_dates=["Дата"])
df.dropna(inplace=True)
df.columns = ["tender.id", "tender.lot.id", "tender.procurementMethod", "tender.date", "tender.value.amount", "tender.items.classification", 
              "tender.mainProcurementCategory", "tender.procuringEntity.identifier.legalName", "tender.procuringEntity.identifier.id", 
              "tender.participant.name", "tender.participant.id", "tender.participant.country",
              "tender.lot.value.amount", "contracts.value.amount", "num_contr_in_tender"]

df = df[df["tender.mainProcurementCategory"]!="Undefined"]

df["contracts.value.amount"] = df["contracts.value.amount"].str.replace(",", ".").astype(float, errors = 'ignore').replace("-", np.NaN)
df["tender.value.amount"] = df["tender.value.amount"].str.replace(",", ".").astype(float, errors = 'ignore').replace("-", np.NaN)
df["tender.lot.value.amount"] = df["tender.lot.value.amount"].str.replace(",", ".").astype(float, errors = 'ignore').replace("-", np.NaN)

print("The shape of the DF: ", df.shape)
df.head(3)

The shape of the DF:  (1206186, 15)


,tender.id,tender.lot.id,tender.procurementMethod,tender.date,tender.value.amount,tender.items.classification,tender.mainProcurementCategory,tender.procuringEntity.identifier.legalName,tender.procuringEntity.identifier.id,tender.participant.name,tender.participant.id,tender.participant.country,tender.lot.value.amount,contracts.value.amount,num_contr_in_tender
1,UA-2016-07-15-000064-c,UA-2016-07-15-000064-c-L1,limited,2020-08-26,8000.0,"09000000-3 Нафтопродукти, паливо, електроенерг...",Goods,Гайсинський медичний фаховий коледж,02011019,Товаристо з обмеженою відповідальністю Торгови...,37149148,Україна,8000.0,7965,1
2,UA-2016-07-15-000064-c,UA-2016-07-15-000064-c-L1,limited,2020-08-26,8000.0,"09000000-3 Нафтопродукти, паливо, електроенерг...",Goods,Гайсинський медичний фаховий коледж,02011019,Товаристо з обмеженою відповідальністю Торгови...,37149148,Україна,8000.0,NaN,1
3,UA-2018-12-29-001589-b,UA-2018-12-29-001589-b-L1,limited,2019-02-01,27358.0,"09000000-3 Нафтопродукти, паливо, електроенерг...",Goods,"КОМУНАЛЬНИЙ ЗАКЛАД КУЛЬТУРИ ""ДНІПРОВСЬКИЙ БУДИ...",37621184,"ТОВ ""ДНІПРОВСЬКІ ЕНЕРГЕТИЧНІ ПОСЛУГИ""",42082379,Україна,27358.0,27358,1


In [131]:
df = df[df["tender.id"].isin(GPA_tenders_list)].reset_index(drop=True)

In [132]:
## Adding the entity type

df_entities = pd.read_excel("Additional data/GPA Довідник.xlsx", sheet_name="Master sheet_All in one", dtype={"EDRPOU":str})
df_entities.drop(["Google translate", "Correct translation"], axis=1, inplace=True)

df = pd.merge(df, df_entities[["EDRPOU", "GPA Kind"]], left_on= "tender.procuringEntity.identifier.id", right_on="EDRPOU", how="left")
df.drop("EDRPOU", axis=1, inplace=True)
df["GPA Kind"].fillna("non-GPA entity", inplace=True)
df = df[df["GPA Kind"]!="non-GPA entity"] ### SIMPLIFY LATER

## Add thresholds
df_thresholds = pd.read_excel("Additional data/Thresholds.xlsx")

df = pd.merge(df, df_thresholds, left_on=["GPA Kind", "tender.mainProcurementCategory"], right_on=["GPA Kind", "CPV type"], how="left")
df.drop("CPV type", axis=1, inplace=True)

## Exchange rate
df_exchange_sdr = pd.read_excel("Additional data/Офіційний курс гривні щодо іноземних валют.xlsx", parse_dates=["Дата"])
df_exchange_sdr["ex_rate"] = df_exchange_sdr["Офіційний курс гривні, грн"] / df_exchange_sdr["Кількість одиниць"]

df = pd.merge(df, df_exchange_sdr[["Дата", "ex_rate"]], left_on= "tender.date", right_on="Дата", how="left")
df.drop("Дата", axis=1, inplace=True)

df["tender.value.amount_SDR"] = df["tender.value.amount"] / df["ex_rate"]
df["tender.lot.value.amount_SDR"] = df["tender.lot.value.amount"] / df["ex_rate"]
df_temp = df[~df["contracts.value.amount"].isnull()]
df_temp["contracts.value.amount_SDR"] = df_temp["contracts.value.amount"].astype(float) / df_temp["ex_rate"]
df = pd.merge(df, df_temp, how="left")

## Finilizing
df = df[df["tender.value.amount_SDR"]>=df["SDR Threshold"]]

df = df[['tender.id', "tender.lot.id", 'tender.procurementMethod', 'tender.date',
       'tender.items.classification',
       'tender.mainProcurementCategory',
       'tender.procuringEntity.identifier.legalName',
       'tender.procuringEntity.identifier.id', 'tender.participant.name',
       'tender.participant.id', 'tender.participant.country',
       'num_contr_in_tender', 'GPA Kind',
       'tender.value.amount_SDR','tender.lot.value.amount_SDR', "contracts.value.amount_SDR"]]

df.rename({"GPA Kind":"GPA entity type"}, axis=1, inplace =True)

print("The shape of the DataFrame: ", df.shape)
df.head(3)

<ipython-input-132-7351340c49a4>:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_temp["contracts.value.amount_SDR"] = df_temp["contracts.value.amount"].astype(float) / df_temp["ex_rate"]


The shape of the DataFrame:  (30538, 16)


,tender.id,tender.lot.id,tender.procurementMethod,tender.date,tender.items.classification,tender.mainProcurementCategory,tender.procuringEntity.identifier.legalName,tender.procuringEntity.identifier.id,tender.participant.name,tender.participant.id,tender.participant.country,num_contr_in_tender,GPA entity type,tender.value.amount_SDR,tender.lot.value.amount_SDR,contracts.value.amount_SDR
0,UA-2018-12-05-002348-c.2,UA-2018-12-05-002348-c.2-L1,selective,2019-03-21,45000000-7 Будівельні роботи та поточний ремонт,Works,Служба автомобільних доріг у Полтавській області,25898491,"ТОВАРИСТВО З ОБМЕЖЕНОЮ ВІДПОВІДАЛЬНІСТЮ ""РОСТД...",33658865,Україна,1,Central government entities,1.795485e+07,1.795485e+07,1.662194e+07
1,UA-2018-12-05-002348-c.2,UA-2018-12-05-002348-c.2-L1,selective,2019-03-21,45000000-7 Будівельні роботи та поточний ремонт,Works,Служба автомобільних доріг у Полтавській області,25898491,"ТОВ ""Шляхове будівництво ""АЛЬТКОМ""",32794511,Україна,1,Central government entities,1.795485e+07,1.795485e+07,NaN
2,UA-2018-12-05-002348-c.2,UA-2018-12-05-002348-c.2-L1,selective,2019-03-21,45000000-7 Будівельні роботи та поточний ремонт,Works,Служба автомобільних доріг у Полтавській області,25898491,ONUR TAAHHUT TASIMACILIK İNSAAT TİCARET VE SAN...,83588,Туреччина,1,Central government entities,1.795485e+07,1.795485e+07,NaN


In [133]:
df["tender.id"].nunique()

7406

In [134]:
df.to_excel("df_participants.xlsx", index=False)

## Complaints

In [ ]:
df = pd.read_excel("Prozorro data/Part_2_Complaints.xls")
df.dropna(inplace=True)
del(df["Count([Ідентифікатор скарги])"])
df.columns = ["tender.id", "complaint.type", "complaint.id", "complaint.submitter.name", "complaint.submitter.country"]

# df = df[df["tender.id"].isin(GPA_tenders_list)].reset_index(drop=True)

print("The shape of the DF: ", df.shape)
df.head(3)

In [ ]:
df.to_excel("df_complaints.xlsx", index=False)

# Calculation

In [136]:
data = pd.read_excel("df_participants.xlsx", dtype={"tender.procuringEntity.identifier.id":str, "tender.participant.id":str})

print("The shape of the DF: ", data.shape)
data.head(3)

The shape of the DF:  (30538, 16)


,tender.id,tender.lot.id,tender.procurementMethod,tender.date,tender.items.classification,tender.mainProcurementCategory,tender.procuringEntity.identifier.legalName,tender.procuringEntity.identifier.id,tender.participant.name,tender.participant.id,tender.participant.country,num_contr_in_tender,GPA entity type,tender.value.amount_SDR,tender.lot.value.amount_SDR,contracts.value.amount_SDR
0,UA-2018-12-05-002348-c.2,UA-2018-12-05-002348-c.2-L1,selective,2019-03-21,45000000-7 Будівельні роботи та поточний ремонт,Works,Служба автомобільних доріг у Полтавській області,25898491,"ТОВАРИСТВО З ОБМЕЖЕНОЮ ВІДПОВІДАЛЬНІСТЮ ""РОСТД...",33658865,Україна,1,Central government entities,1.795485e+07,1.795485e+07,1.662194e+07
1,UA-2018-12-05-002348-c.2,UA-2018-12-05-002348-c.2-L1,selective,2019-03-21,45000000-7 Будівельні роботи та поточний ремонт,Works,Служба автомобільних доріг у Полтавській області,25898491,"ТОВ ""Шляхове будівництво ""АЛЬТКОМ""",32794511,Україна,1,Central government entities,1.795485e+07,1.795485e+07,NaN
2,UA-2018-12-05-002348-c.2,UA-2018-12-05-002348-c.2-L1,selective,2019-03-21,45000000-7 Будівельні роботи та поточний ремонт,Works,Служба автомобільних доріг у Полтавській області,25898491,ONUR TAAHHUT TASIMACILIK İNSAAT TİCARET VE SAN...,83588,Туреччина,1,Central government entities,1.795485e+07,1.795485e+07,NaN


In [137]:
df = data[["tender.id", "tender.procurementMethod", "tender.date", "tender.value.amount_SDR", "tender.mainProcurementCategory"]].drop_duplicates()

print("The shape of the DF: ", df.shape)
df.head(3)

The shape of the DF:  (7406, 5)


,tender.id,tender.procurementMethod,tender.date,tender.value.amount_SDR,tender.mainProcurementCategory
0,UA-2018-12-05-002348-c.2,selective,2019-03-21,1.795485e+07,Works
3,UA-2018-12-26-003794-b,limited,2019-11-01,1.319491e+05,Services
8,UA-2018-12-28-000557-a,limited,2019-08-01,3.634378e+06,Goods


In [138]:
pd.pivot_table(data=df, values="tender.value.amount_SDR", 
               index="tender.procurementMethod", columns=df["tender.date"].dt.year, 
               aggfunc=["sum", "count"])

sum                             count  \
tender.date                       2019          2020          2021  2019   
tender.procurementMethod                                                   
limited                   9.554926e+08  8.218097e+08  8.993458e+08   721   
open                      3.581727e+09  4.539644e+09  6.021178e+09  1770   
selective                 8.974706e+07  1.215386e+08  1.693295e+08     4   

                                      
tender.date               2020  2021  
tender.procurementMethod              
limited                    605   641  
open                      1841  1685  
selective                   66    73

In [139]:
pd.pivot_table(data=df, values="tender.value.amount_SDR", 
               index="tender.mainProcurementCategory", columns=df["tender.date"].dt.year, 
               aggfunc=["sum", "count"])

sum                              \
tender.date                             2019          2020          2021   
tender.mainProcurementCategory                                             
Goods                           1.880024e+09  1.708392e+09  1.772859e+09   
Services                        1.095097e+09  1.267161e+09  7.907664e+08   
Works                           1.651846e+09  2.507440e+09  4.526228e+09   

                               count              
tender.date                     2019  2020  2021  
tender.mainProcurementCategory                    
Goods                           1620  1602  1452  
Services                         774   743   669  
Works                            101   167   278

In [151]:
pd.concat([data[["tender.id", "tender.value.amount_SDR", "tender.date"]].drop_duplicates().groupby(data["tender.date"].dt.year).sum(),
           data[["tender.id", "tender.lot.id", "tender.lot.value.amount_SDR", "tender.date"]].drop_duplicates().groupby(data["tender.date"].dt.year).sum(),
           data["contracts.value.amount_SDR"].groupby(data["tender.date"].dt.year).sum()], axis=1)

,tender.value.amount_SDR,tender.lot.value.amount_SDR,contracts.value.amount_SDR
tender.date,,,
2019,4.626967e+09,4.561029e+09,3.993350e+09
2020,5.482993e+09,5.349154e+09,4.869068e+09
2021,7.089853e+09,7.038635e+09,6.683158e+09


In [85]:
df["tender.value.amount_SDR"].groupby(df["tender.date"].dt.to_period("Q")).agg(["sum", "count"])

,sum,count
tender.date,,
2019Q1,1.322215e+09,715
2019Q2,8.899958e+08,569
2019Q3,6.876167e+08,418
2019Q4,1.727139e+09,793
2020Q1,1.021237e+09,726
2020Q2,8.940294e+08,553
2020Q3,1.200873e+09,502
2020Q4,2.366853e+09,731
2021Q1,1.730248e+09,661


In [96]:
df = data[["tender.id", "tender.value.amount_SDR", "tender.date"]].drop_duplicates()

df["with_ foreigner"] = df["tender.id"].isin(data["tender.id"][data["tender.participant.country"]!="Україна"].unique())

print("The shape of the DF: ", df.shape)
df.head(3)

The shape of the DF:  (7406, 4)


,tender.id,tender.value.amount_SDR,tender.date,with_ foreigner
0,UA-2018-12-05-002348-c.2,1.795485e+07,2019-03-21,True
3,UA-2018-12-26-003794-b,1.319491e+05,2019-11-01,False
8,UA-2018-12-28-000557-a,3.634378e+06,2019-08-01,False


In [97]:
pd.pivot_table(data=df, values="tender.value.amount_SDR", 
               index="with_ foreigner", columns=df["tender.date"].dt.year, 
               aggfunc=["sum", "count"]).append(

pd.pivot_table(data=df, values="tender.value.amount_SDR", 
               index="with_ foreigner", columns=df["tender.date"].dt.year, 
               aggfunc=["sum", "count"]) / 
 pd.pivot_table(data=df, values="tender.value.amount_SDR", 
               index="with_ foreigner", columns=df["tender.date"].dt.year, 
               aggfunc=["sum", "count"]).sum()
).drop(False)

sum                                   count  \
tender.date              2019          2020          2021        2019   
with_ foreigner                                                         
True             7.764122e+08  1.236643e+09  6.112915e+08  107.000000   
True             1.678015e-01  2.255417e-01  8.622062e-02    0.042886   

                                         
tender.date            2020        2021  
with_ foreigner                          
True             133.000000  115.000000  
True               0.052946    0.047937

In [154]:
df = data[["tender.id", "tender.lot.id", "tender.date", "tender.participant.country","contracts.value.amount_SDR"]].dropna(subset=["contracts.value.amount_SDR"])

# df["with_ foreigner"] = df["tender.id"].isin(data["tender.id"][data["tender.participant.country"]!="Україна"].unique())

print("The shape of the DF: ", df.shape)
df.head(3)

The shape of the DF:  (11986, 5)


,tender.id,tender.lot.id,tender.date,tender.participant.country,contracts.value.amount_SDR
0,UA-2018-12-05-002348-c.2,UA-2018-12-05-002348-c.2-L1,2019-03-21,Україна,1.662194e+07
3,UA-2018-12-26-003794-b,UA-2018-12-26-003794-b-L3,2019-11-01,Україна,3.036803e+03
4,UA-2018-12-26-003794-b,UA-2018-12-26-003794-b-L2,2019-11-01,Україна,7.166856e+04


In [160]:
pd.pivot_table(data=df, values="contracts.value.amount_SDR", 
               index=df["tender.participant.country"]!="Україна", columns=df["tender.date"].dt.year, 
               aggfunc=["sum", "count"]).append(

pd.pivot_table(data=df, values="contracts.value.amount_SDR", 
               index=df["tender.participant.country"]!="Україна", columns=df["tender.date"].dt.year, 
               aggfunc=["sum", "count"]) / 
 pd.pivot_table(data=df, values="contracts.value.amount_SDR", 
               index=df["tender.participant.country"]!="Україна", columns=df["tender.date"].dt.year, 
               aggfunc=["sum", "count"]).sum()
).drop(False)

sum                              \
tender.date                         2019          2020          2021   
tender.participant.country                                             
True                        5.914491e+08  6.455420e+08  1.349920e+08   
True                        1.481085e-01  1.325802e-01  2.019883e-02   

                                count                       
tender.date                      2019      2020       2021  
tender.participant.country                                  
True                        83.000000  87.00000  76.000000  
True                         0.020373   0.02037   0.020873

In [118]:
df = pd.pivot_table(data=data, values="tender.participant.id",
              index="tender.participant.country", columns=data["tender.date"].dt.year,
              aggfunc="nunique").fillna(0)

df.sort_values(by=2021, ascending=False).head(10)

tender.date,2019,2020,2021
tender.participant.country,,,
Україна,2514.0,2586.0,2295.0
Велика Британія,6.0,7.0,11.0
Чехія,9.0,9.0,9.0
Туреччина,4.0,6.0,7.0
Федеративна Республіка Німеччина,6.0,9.0,6.0
Китайська Народна Республіка,1.0,6.0,5.0
Словаччина,3.0,3.0,4.0
Латвія,5.0,2.0,3.0
Швейцарія,4.0,3.0,3.0
